# File binary operations

File binary operations are essential for handling the persistence of binary data.
Python allows the basic operations of opening and closing a binary file, along with reading and writing binary data into it.

## Persisting binary data
Data can be persisted not only in text format, but also in binary format by using Python.

In [1]:
import struct

# creating a complex data instance
product_instance = {
    "name": "Personal Laptop",
    "sku": "PX23332223",
    "cost_EUR": 1000.50,
    "is_available": False
}

# creating the binary form of product data instance
def pack_product_data(product):
    # calculate packing parameters
    len_name = len(product["name"])
    len_sku = len(product["sku"])
    format_string = "q q q {0}s {1}s f ?".format(len_name, len_sku)
    length_data = struct.calcsize(format_string)

    # pack data using the calculated format string
    packed_data = struct.pack(
        format_string,
        # product header data containing various data lengths
        length_data,
        len_name,
        len_sku,
        # product effective data
        bytearray(product["name"], encoding="utf-8"),
        bytearray(product["sku"], encoding="utf-8"),
        product["cost_EUR"],
        product["is_available"]
    )

    return packed_data


product_instance_packed_data = pack_product_data(product_instance)

In [2]:
# creating the file for writing data, it must have the 'b' flag for writing binary data.
# it should use 'w' for overwriting content in a new file 
# and 'a' for appending content to an existing file
data_file = open("product.data", "wb")
data_file.write(product_instance_packed_data)
data_file.close()

## Reading binary data

Data stored in binary files can be used to restore persisted objects and continue to be processed by applications.

In [3]:
# creating the file for writing data, it must have the 'b' flag for writing binary data.
# it should use 'r' for reading content
data_file = open("product.data", "rb")
product_instance_read_data = data_file.read()
data_file.close()

In [4]:
# the unpack method allows to convert back data from binary format
def unpack_product_data(byte_array):
    # read the packing header
    product_info_header_size = struct.calcsize("q q q")
    data_size, name_size, sku_size = struct.unpack("q q q", byte_array[0:product_info_header_size])
    
    # calculate the product data parameters
    format_string = "{0}s {1}s f ?".format(name_size, sku_size)
    product_data_size = struct.calcsize(format_string)

    # unpack data and create the product instance
    product_byte_data = byte_array[product_info_header_size: product_info_header_size + product_data_size]
    name, sku, cost_EUR, is_available = struct.unpack(format_string, product_byte_data)
    product = {
        "name": name.decode("UTF-8"),
        "sku": sku.decode("UTF-8"),
        "cost_EUR": cost_EUR,
        "is_available": is_available
    }
    
    return data_size, product

data_size, product = unpack_product_data(product_instance_read_data)

print("The unpacked instance data is:\n{0} .".format(product))

The unpacked instance data is:
{'name': 'Personal Laptop', 'sku': 'PX23332223', 'cost_EUR': 1000.5, 'is_available': False} .
